In [5]:
# Disable jedi autocompleter
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import gym

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from agents import ExpertPIDController

In [4]:
test_controller = ExpertPIDController(1, 2)

In [ ]:
controller_action, f1_vels, f2_vels, f3_vels, wrist_vels = controller.PDController(lift_check, obs, env.action_space, velocities)


velocities = {"constant_velocity": 0.75, "min_velocity": 0, "max_velocity": 1.5, "finger_lift_velocity": 1, "wrist_lift_velocity": 1}



In [ ]:
def get_action(obs, lift_check, controller, env, velocities, pid_mode="combined",timestep=None):
    """ Get action based on controller (Naive, position-dependent, combined interpolation)
        obs: Current state observation
        controller: Initialized expert PID controller
        env: Current Mujoco environment needed for expert PID controller
        return action: np.array([wrist, f1, f2, f3]) (velocities in rad/sec)
    """
    object_x_coord = obs[21]  # Object x coordinate position

    # By default, action is set to close fingers at a constant velocity
    controller_action = np.array([velocities["constant_velocity"], velocities["constant_velocity"], velocities["constant_velocity"]])

    # NAIVE CONTROLLER: Close all fingers at a constant speed
    if pid_mode == "naive":
        controller_action = NaiveController(lift_check, velocities)

    # POSITION-DEPENDENT CONTROLLER: Only move fingers based on object x-coord position within hand
    elif pid_mode == "position-dependent":
        controller_action, f1_vels, f2_vels, f3_vels, wrist_vels = controller.PDController(lift_check, obs, env.action_space, velocities)

    elif pid_mode == "bell-shaped":
        controller_action = BellShapedController(lift_check, velocities, timestep)

    # COMBINED CONTROLLER: Interpolate Naive and Position-Dependent controller output based on object x-coord position within hand
    else:
        # If object x position is on outer edges, do expert pid
        if object_x_coord < -0.04 or object_x_coord > 0.04:
            # Expert Nudge controller strategy
            controller_action, f1_vels, f2_vels, f3_vels, wrist_vels = controller.PDController(lift_check, obs, env.action_space, velocities)

        # Object x position within the side-middle ranges, interpolate expert/naive velocity output
        elif -0.04 <= object_x_coord <= -0.02 or 0.02 <= object_x_coord <= 0.04:
            # Interpolate between naive and expert velocities
            # position-dependent controller action (finger velocity based on object location within hand)
            expert_action, f1_vels, f2_vels, f3_vels, wrist_vels = controller.PDController(lift_check, obs, env.action_space, velocities)
            # Naive controller action (fingers move at constant velocity)
            naive_action = NaiveController(lift_check, velocities)

            # Interpolate finger velocity values between position-dependent and Naive action output
            finger_vels = np.interp(np.arange(0, 3), naive_action, expert_action)

            controller_action = np.array([finger_vels[0], finger_vels[1], finger_vels[2]])

        # Object x position is within center area, so use naive controller
        else:
            # Naive controller action (fingers move at constant velocity)
            controller_action = NaiveController(lift_check, velocities)

    #print("**** action: ",action)

    return controller_action

In [ ]:
# First, get the observation

# Then, get the action from controller
# Input required: 1. If we are done lifting (hard code a certain amount of timesteps and just use done flag?), 2. observation, 3. action space (shoudl jsut be highs and lows), velocities dictionary
# todo: rewrite so we don't feed in specific values
controller_action, f1_vels, f2_vels, f3_vels, wrist_vels = test_controller.PDController(lift_check, obs, env.action_space, velocities)

# Within here:
# First, calculate the distances


# Then, take the action

In [ ]:
env = gym.make('gym_kinova_gripper:kinovagripper-v0')

        start_hand_rotation = [-1.57 + roll_noise, 0 + pitch_noise, -1.57 + yaw_noise]

        obs = env.reset(shape_keys=['CubeM'], hand_orientation='normal',
                        start_pos=[x_noise, y_noise, z_noise],
                        hand_rotation=start_hand_rotation)  # TODO: make sure this is NO OBJECT
        